In [45]:
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

z1_xml_file = './FRB_z1/Z1_data.xml'
schema_file = './FRB_z0/Z1_Z1.xsd'


# Define the namespaces

namespaces = {
    'kf': 'http://www.federalreserve.gov/structure/compact/Z1_Z1',
    'frb': 'http://www.federalreserve.gov/structure/compact/common',
    'common': 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/common'
    
}
# Parse the XML file
tree = ET.parse(z1_xml_file)
root = tree.getroot()

# Iterate through kf:Series elements
all_series = []

column_names =['Date','Obs_value','Series_name']

z1df = pd.DataFrame(columns=column_names)

# Find all kf:Series elements
series_elements = root.findall('.//kf:Series', namespaces)

# Iterate through kf:Series elements with a progress bar
for series in tqdm(series_elements, desc="Processing series"):
    series_attributes = series.attrib
    
    # Extract frb:Annotations data
    annotations_element = series.find('frb:Annotations', namespaces)
    #print("Annotations Element:", annotations_element)  # Debug print
    if annotations_element is None:
        print('Error: Annotations element not found')
        continue
    
    for annotation in annotations_element.findall('common:Annotation', namespaces):
        annotation_type = annotation.find('common:AnnotationType', namespaces).text
        annotation_text = annotation.find('common:AnnotationText', namespaces).text
        series_attributes[annotation_type] = annotation_text

     # Iterate through frb:Obs elements within each kf:Series (if needed)
    obs_value=[]
    time_period=[]
    for obs in series.findall('frb:Obs', namespaces):
        obs_value.append(obs.get('OBS_VALUE'))
        time_period.append(obs.get('TIME_PERIOD'))

    temp_dic = {'Date':time_period,'Obs_value':obs_value}
    tempdf= pd.DataFrame(temp_dic)
    tempdf['Series_name'] = series_attributes['SERIES_NAME'] 
    
    all_series.append(series_attributes)
    
    # Stack the temporary DataFrame to the main DataFrame
    z1df = pd.concat([z1df, tempdf], ignore_index=True)

    
    #print("Updated Series Attributes:", series_attributes)  # Print updated series attributes

# show Z1df

z1df.head()

KeyboardInterrupt: 

In [2]:
import pandas as pd

# Load the Parquet file into a pandas DataFrame
z1df = pd.read_parquet('z1df.parquet')

# Display the first few rows of the DataFrame
z1df.head()

,Date,Obs_value,Series_name
0,1946-12-31,201261.0,FA086010005.A
1,1947-12-31,218742.0,FA086010005.A
2,1948-12-31,244849.0,FA086010005.A
3,1949-12-31,239729.0,FA086010005.A
4,1950-12-31,266613.0,FA086010005.A


In [6]:
filtered_df = z1df[z1df['Series_name'].str.contains('LM893064105.Q', na=False)]
filtered_df.head()


,Date,Obs_value,Series_name
4277688,1945-12-31,152852.0,LM893064105.Q
4277689,1946-03-31,-9999.0,LM893064105.Q
4277690,1946-06-30,-9999.0,LM893064105.Q
4277691,1946-09-30,-9999.0,LM893064105.Q
4277692,1946-12-31,146656.0,LM893064105.Q


In [18]:
# Assuming 'Date' is a column in your DataFrame
# Pivot the DataFrame
pivoted_df = filtered_df.pivot(index='Date', columns='Series_name', values='Obs_value')

# Select only the 'LM893064105.Q' column
pivoted_df = pivoted_df[['LM893064105.Q']]

# Reset the index to have 'Date' as a column
pivoted_df.reset_index(inplace=True)



# Reset the index again to add a simple integer index
pivoted_df.reset_index(drop=True, inplace=True)

# Rename the index to 'index'
#pivoted_df.rename_axis('index', inplace=True)



# Display the pivoted DataFrame
print(pivoted_df.head())

Series_name       Date  LM893064105.Q
0           1945-12-31       152852.0
1           1946-03-31        -9999.0
2           1946-06-30        -9999.0
3           1946-09-30        -9999.0
4           1946-12-31       146656.0


In [19]:
series_dic = {
        "LM153064105": "Household sector",
        "LM103064103": "Nonfinancial corporate business",
        "LM313064105": "Federal government",
        "LM213064103": "State and local governments",
        "FL713064103": "Monetary authority",
        "LM763064103": "U.S.-chartered depository institutions",
        "FL753064103": "Foreign banking offices in U.S.",
        "LM513064105": "Property-casualty insurance companies",
        "LM543064105": "Life insurance companies",
        "LM573064105": "Private pension funds",
        "LM343064105": "Federal government retirement funds",
        "LM223064145": "State and local govt. retirement funds",
        "LM653064100": "Mutual funds",
        "LM553064103": "Closed-end funds",
        "LM563064100": "Exchange-traded funds",
        "LM663064103": "Brokers and dealers",
        "FL503064105": "Other financial business",
        "LM263064105": "Rest of the world",
    }

In [22]:
series_names = [x + '.Q' for  x in list(series_dic.keys())]

equity_holder = z1df[z1df['Series_name'].isin(series_names)]
    

In [23]:
equity_holder.head()

,Date,Obs_value,Series_name
1486130,1945-12-31,0.0,FL503064105.Q
1486131,1946-03-31,-9999.0,FL503064105.Q
1486132,1946-06-30,-9999.0,FL503064105.Q
1486133,1946-09-30,-9999.0,FL503064105.Q
1486134,1946-12-31,0.0,FL503064105.Q


In [25]:
# Assuming 'Date' is a column in your DataFrame
# Pivot the DataFrame
pivoted_equity_holder = equity_holder.pivot(index='Date', columns='Series_name', values='Obs_value')

# Select only the 'LM893064105.Q' column
pivoted_equity_holder = pivoted_equity_holder[series_names]

# Reset the index to drop the current index and convert it to columns
pivoted_equity_holder.reset_index(inplace=True)

# Reset the index again to add a simple integer index
pivoted_equity_holder.reset_index(drop=True, inplace=True)

# Rename the index to 'index'
pivoted_equity_holder.rename_axis('index', inplace=True)

# Display the pivoted DataFrame
pivoted_equity_holder.head()

Series_name,Date,LM153064105.Q,LM103064103.Q,LM313064105.Q,LM213064103.Q,FL713064103.Q,LM763064103.Q,FL753064103.Q,LM513064105.Q,LM543064105.Q,LM573064105.Q,LM343064105.Q,LM223064145.Q,LM653064100.Q,LM553064103.Q,LM563064100.Q,LM663064103.Q,FL503064105.Q,LM263064105.Q
index,,,,,,,,,,,,,,,,,,,
0,1945-12-31,144855.0,0.0,0.0,0.0,0.0,166.0,1.0,1760.0,999.0,0.0,0.0,7.0,1033.0,777.0,0.0,554.0,0.0,2700.0
1,1946-03-31,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,1946-06-30,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,1946-09-30,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,1946-12-31,138515.0,0.0,0.0,0.0,0.0,180.0,1.0,1719.0,1249.0,0.0,0.0,8.0,1044.0,800.0,0.0,450.0,0.0,2690.0


In [29]:
print(pivoted_df.shape[0])
print(pivoted_equity_holder.shape[0])

314
314


In [37]:
import numpy as np

# Assuming both DataFrames have a 'Date' column
merged_df = pivoted_equity_holder.merge(pivoted_df, on='Date')


# Replace -9999.0 with NaN in the merged DataFrame
merged_df.replace(-9999.0, np.nan, inplace=True)


# Display the merged DataFrame
merged_df.head()

Series_name,Date,LM153064105.Q,LM103064103.Q,LM313064105.Q,LM213064103.Q,FL713064103.Q,LM763064103.Q,FL753064103.Q,LM513064105.Q,LM543064105.Q,LM573064105.Q,LM343064105.Q,LM223064145.Q,LM653064100.Q,LM553064103.Q,LM563064100.Q,LM663064103.Q,FL503064105.Q,LM263064105.Q,LM893064105.Q
0,1945-12-31,144855.0,0.0,0.0,0.0,0.0,166.0,1.0,1760.0,999.0,0.0,0.0,7.0,1033.0,777.0,0.0,554.0,0.0,2700.0,152852.0
1,1946-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1946-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1946-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1946-12-31,138515.0,0.0,0.0,0.0,0.0,180.0,1.0,1719.0,1249.0,0.0,0.0,8.0,1044.0,800.0,0.0,450.0,0.0,2690.0,146656.0


In [38]:
# Assuming 'series_names' is a list of column names in merged_df
series_names = [col for col in merged_df.columns if col not in ['Date', 'LM893064105.Q']]

# Divide each series in series_names by the 'LM893064105.Q' column
for series in series_names:
    merged_df[series] = merged_df[series] / merged_df['LM893064105.Q'] *100

# Display the modified DataFrame
merged_df.head()

Series_name,Date,LM153064105.Q,LM103064103.Q,LM313064105.Q,LM213064103.Q,FL713064103.Q,LM763064103.Q,FL753064103.Q,LM513064105.Q,LM543064105.Q,LM573064105.Q,LM343064105.Q,LM223064145.Q,LM653064100.Q,LM553064103.Q,LM563064100.Q,LM663064103.Q,FL503064105.Q,LM263064105.Q,LM893064105.Q
0,1945-12-31,94.768142,0.0,0.0,0.0,0.0,0.108602,0.000654,1.151441,0.653573,0.0,0.0,0.004580,0.675817,0.508335,0.0,0.362442,0.0,1.766415,152852.0
1,1946-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1946-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1946-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1946-12-31,94.448914,0.0,0.0,0.0,0.0,0.122736,0.000682,1.172131,0.851653,0.0,0.0,0.005455,0.711870,0.545494,0.0,0.306840,0.0,1.834224,146656.0


In [39]:
import plotly.express as px

# Assuming 'series_names' is a list of column names in merged_df
series_names = [col for col in merged_df.columns if col not in ['Date', 'LM893064105.Q']]

# Melt the DataFrame to long format for Plotly
melted_df = merged_df.melt(id_vars=['Date'], value_vars=series_names, var_name='Series', value_name='Value')

# Create the area plot
fig = px.area(melted_df, x='Date', y='Value', color='Series', title='Area Plot of Series')

# Show the plot
fig.show()

In [40]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming 'series_names' is a list of column names in merged_df
series_names = [col for col in merged_df.columns if col not in ['Date', 'LM893064105.Q']]

# Create a subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add area traces for each series in series_names
for series in series_names:
    fig.add_trace(
        go.Scatter(x=merged_df['Date'], y=merged_df[series], mode='lines', stackgroup='one', name=series),
        secondary_y=False,
    )

# Add a line trace for 'LM893064105.Q' on the secondary y-axis
fig.add_trace(
    go.Scatter(x=merged_df['Date'], y=merged_df['LM893064105.Q'], mode='lines', name='LM893064105.Q', line=dict(color='red')),
    secondary_y=True,
)

# Update layout
fig.update_layout(
    title='Area Plot of Series with Secondary Y-Axis for LM893064105.Q',
    xaxis_title='Date',
    yaxis_title='Series Values',
    yaxis2_title='LM893064105.Q',
)

# Show the plot
fig.show()